### Extracting the Data

In [1]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

USER_ID = '227dt3g18rrzyco2j7p2boy2ly'
TOKEN = 'BQDrII1YwKUcT-bGt5GBwJgoU4Q_mpqoQLlkNDH8m3xTIKuu6j-wNHARndlZy4nqBF3GIDnSec85a0-Z4mrukDNIASszsASuyzFjfqIgOji3FW-7VwHshUDDVq8Wg-jukevB11E4PjLhqq4VDEw9NdsSO5xVyvh_8vsa5lrC'

In [2]:
def data_validation(df:pd.DataFrame):
#is the datadrame empty?    
    if df.empty:
        print("No songs were listened in the last 24 hours")
        return False

#are primary keys ok?

    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("The primary key conditions has been corrupted")

#are null values at song_id
    
    if df.isnull().values['song_id']:
        raise Exception("There is a Null Value in the played_at column")
    else:
        return True
    
        

In [3]:
headers = {
    "Accept":"application/json",
    "Content-Type":"application/json",
    "Authorization":"Bearer {token}".format(token=TOKEN)
}
    

In [4]:
r = requests.get("https://api.spotify.com/v1/me/player/recently-played",headers = headers)
response = r.json()

In [5]:
my_song_list = []

for song in response['items']:
    artist_id = song['track']['artists'][0]['id']
    artist_name = song['track']['artists'][0]['name']
    artist_link = song['track']['artists'][0]['external_urls']['spotify'] 
    album_id = song['track']['album']['id']
    album_name = song['track']['album']['name']
    album_link = song['track']['album']['external_urls']['spotify']
    song_id = song['track']['id']
    song_name = song['track']['name']
    song_link = song['track']['external_urls']['spotify']
    duration_ms = song['track']['duration_ms']
    popularity = song['track']['popularity']
    disc_number = song['track']['disc_number']
    played_at = song['played_at'].split(".")[0]
    
    song_dic = {'artist_id': artist_id,
                    'artist_name':artist_name,
                    'artist_link':artist_link,
                    'album_id':album_id,
                    'album_name':album_name,
                    'album_link':album_link,
                    'song_id':song_id,
                    'song_name':song_name,
                    'song_link':song_link,
                    'duration_ms':duration_ms,
                    'popularity':popularity,
                    'disc_number':disc_number,
                    'played_at':played_at
                    }
    
    my_song_list.append(song_dic)
    df = pd.DataFrame(my_song_list)


### Cleaning Up the Data

In [6]:
df = df[["artist_id","artist_link","album_id","album_name","album_link","song_id","song_name","song_link","duration_ms","popularity","disc_number","played_at"]]
df[['date','time']] = df['played_at'].str.split('T',expand=True)
#df = df.drop(columns=['played_at'])


In [9]:
if data_validation(df):
    print("Data validation ran succesfully")


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices